## **| 모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**

확률적 경사하강법, 미니배치 경사 하강법을 사용할 수 있다.
정규방정식의 경우 계산 복잡도가 n에 따라 가파르게 증가하기 때문에 사용하기에 적절하지 않다

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
학습률이 너무 높아 알고리즘이 발산하고 있을 가능성이 높다. 오버피팅의 문제이므로 검증 에러가 최솟값에 도달했을 때 학습을 조기종료하며 해결할 수 있다.

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
훈련 오차와 검증 오차가 비슷하고 둘다 높다면 모델이 훈련 세트에 과소적합 되었을 가능성이 있다. 높은 편향을 가진 모델이므로 규제 하이퍼 파라미터를 감소시켜야한다.

### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀

규제가 있는 모델이 규제가 없는 모델에 비해 성능이 좋기 때문이다.
- 릿지 회귀 대신 라쏘 회귀

라쏘 회귀는 패널티를 통해 작은 가중치를 0으로 만드는 경향이 있다. 가장 중요한 가중치를 제외하고는 모두 0으로 만들기 때문에 희소한 모델을 만든다. 이는 자동 특성 선택의 효과를 가지므로 몇개의 특정한 값만 실제 유용할 것이라고 판단될 때 사용하기 적합하다.
- 라쏘 회귀 대신 엘라스틱넷

특성이 몇개 강하게 연관되어 있거나 n이 훈련 샘플보다 많을 때 라쏘가 불규칙적으로 행동될 수 있기 때문에 엘라스틱넷을 사용한다. 

### **추가) 조기 종료를 사용한 배치 경사 하강법으로 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**
---



In [1]:
from sklearn import datasets
iris=datasets.load_iris()
X=iris['data'][:,(2,3)]
y=iris['target']

C:\Users\MYCOM\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
#X와 1 shape의 array 와 cbind 하기
import numpy as np
X_b=np.c_[np.ones([len(X),1]),X]
np.random.seed(1)
# 랜덤 시드 고정
# train_test_split 만들기 
test_ratio=0.2
validation_ratio=0.2
total_size=len(X_b)
test_size=int(total_size*test_ratio)
validation_size=int(total_size*validation_ratio)
train_size=total_size-test_size-validation_size
# 무작위 순서 바꾸기
rn_idx=np.random.permutation(total_size)
X_train=X_b[rn_idx[:train_size]]
y_train=y[rn_idx[:train_size]]
X_valid=X_b[rn_idx[train_size:-test_size]]
y_valid=y[rn_idx[train_size:-test_size]]
X_test=X_b[rn_idx[-test_size:]]
y_test=y[rn_idx[-test_size:]]

In [8]:
y_test

array([0, 2, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 2, 0, 0, 2,
       1, 2, 1, 2, 2, 1, 2, 0])

In [10]:
# 소프트맥스 회귀 모델을 훈련시키기 위해선 타깃 클래스의 확률을 알아야 함
# 확률이 1인 타깃 클래스를 제외한 다른 클래스의 확률은 0
# one_hot 벡터로 바꿔주기
def one_hot(y):
    n_classes=y.max()+1
    m=len(y)
    Y_onehot=np.zeros((m,n_classes))
    Y_onehot[np.arange(m),y]=1
    return Y_onehot

In [11]:
# Y 값 모두 바꿔주기

y_train_onehot=one_hot(y_train)
y_test_onehot=one_hot(y_test)
y_valid_onehot=one_hot(y_valid)

In [13]:
y_test_onehot

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [14]:
# 소프트맥스 함수 쓰기
def softmax(x):
    exp=np.exp(x)
    exp_s=np.sum(exp,axis=1,keepdims=True)
    return exp/exp_s

In [15]:
X_train.shape

(90, 3)

In [16]:
len(np.unique(y_train))

3

In [22]:
# 크로스 엔트로피 비용함수
eta=0.01
n_iterations=5000
m=len(X_train)
alpha=0.1
best_loss=np.infty #조기 종류 추가 
Theta=np.random.randn(3,3)
for iteration in range(n_iterations):
    logits=X_train.dot(Theta)
    y_proba=softmax(logits)
    entropy_loss=-np.mean(np.sum(y_train_onehot*np.log(y_proba+1e-7),axis=1))
    l2_loss=1/2*np.sum(np.square(Theta[1:]))
    loss=entropy_loss+alpha*l2_loss
    error=y_proba-y_train_onehot
    gradients=1/m*X_train.T.dot(error)+np.r_[np.zeros([1,3]),alpha*Theta[1:]]
    Theta=Theta-eta*gradients
    logits=X_valid.dot(Theta)
    y_proba=softmax(logits)
    entropy_loss=-np.mean(np.sum(y_valid_onehot*np.log(y_proba+1e-7),axis=1))
    l2_loss=1/2*np.sum(np.square(Theta[1:]))
    loss=entropy_loss+alpha*l2_loss
    if iteration%500==0:
        print(iteration,loss)
    if loss<best_loss:
        best_loss=loss
    else:
        print(iteration-1,best_loss)
        print(iteration,loss,'Early Stopping-!')
        break

0 3.339528736087666
500 0.9365396657020225
1000 0.7516536811842142
1500 0.6608878343187011
2000 0.6119789827398047
2500 0.5829349541862657
3000 0.5640900120092367
3500 0.5509438596703224
4000 0.5412512816873222
4500 0.5338052539685751
